# Fast low-rank nonnegative tensor factorizations in tensor train format

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import scipy.io as spio
from time import time

from alternating_projections import NTTSVD
from tensor_train import TensorTrain

In [2]:
%%bash
uname -m
sysctl -n machdep.cpu.brand_string
echo -n "CPU(s): " && sysctl -n hw.physicalcpu
echo -n "RAM:    " && sysctl -n hw.memsize | awk '{print $0/1073741824" GB"}'

x86_64
Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
CPU(s): 6
RAM:    16 GB


# 1 Hilbert

In [3]:
sizes = [128, 128, 128]
ranks = [1, 3, 2, 1]

a = np.empty(shape=sizes)

m, n, l = a.shape
for i in range(m):
    for j in range(n):
        for k in range(l):
            a[i, j, k] = 1 / ((i+1) + (j+1) + (k+1) - 3 + 1)

In [4]:
# negative elements after TTSVD
full = TensorTrain.GetFullTensor(TensorTrain.TTSVD(a, ranks))
print('negative elements (frobenius):', np.linalg.norm(full[full < 0]))
print('negative elements (density):', (full < 0).sum() / np.prod(full.shape))
print('min element:', full.min())

negative elements (frobenius): 0.09767864271860334
negative elements (density): 6.341934204101562e-05
min element: -0.016309723185813894


In [5]:
# NTTSVD

t0 = time()

cores = NTTSVD(a, ranks=ranks, itersNum=250)

print(time() - t0, 's.')
full = TensorTrain.GetFullTensor(cores=cores)
print('relative error:', np.linalg.norm(a - full) / np.linalg.norm(a))
print('negative elements (frobenius):', np.linalg.norm(full[full < 0]))
print('negative elements (density):', (full < 0).sum() / np.prod(full.shape))

49.58812594413757 s.
relative error: 0.07882071085890664
negative elements (frobenius): 8.624020822013567e-16
negative elements (density): 1.9073486328125e-06


In [6]:
# fast_nlrt

t0 = time()

tt = TensorTrain(cores=TensorTrain.TTSVD(a, ranks))
minElement = tt.GetMinElement()

cores = []
for p in range(len(tt.GetSizes())):
    cores.append(np.ones((1, tt.GetSizes()[p], 1)))
cores[0] *= minElement
ttWithRanks1 = TensorTrain(cores=cores)
tt = tt + ttWithRanks1
print(time() - t0, 's.')

full = TensorTrain.GetFullTensor(cores=tt.GetCores())
print('relative error:', np.linalg.norm(a - full) / np.linalg.norm(a))
print('negative elements (frobenius):', np.linalg.norm(full[full < 0]))
print('negative elements (density):', (full < 0).sum() / np.prod(full.shape))

23.16606307029724 s.
relative error: 0.2978183768899082
negative elements (frobenius): 0.08002339083190409
negative elements (density): 4.8160552978515625e-05


In [7]:
# minElement (ALS)
minElement

0.0021318819125355688